In [ ]:
import locale
import traceback
import pymysql
import dash
from dash import html
from dash.dependencies import Input, Output, State
import rol_usuario_funciones as ruf
import rol_usuario_funciones_filtros as ruff
import rol_administrador_inserciones_funciones as raif
import rol_administrador_gestion_funciones as ragf
import layout as ly 
import login as log

# Setea la variable LC_ALL al conjunto de código UTF-8 con descripción español España
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

colors = {"graphBackground": "#F5F5F5", "background": "#ffffff", "text": "#000000"}

user_logged = ""


Creamos el layout.

In [ ]:
app.layout =  html.Div(
            children=[
               ly.generar_layout_usuario(),
               ly.generar_layout_login(),
               ly.generar_layout_administrador(),
               ly.generar_layout_elegir_rol()
            ],
        )       

Login, si entra como usuario, cargamos el dataframe y las listas desplegables

In [ ]:
# Controlamos el login y las visibilidades según el rol del usuario
# Cargamos informacion para opciones si entramos como usuario
@app.callback(
    Output("layout-login", "style"),
    Output("layout-usuario", "style"),
    Output("layout-administrador", "style"),
    Output("layout-elegir-rol", "style"),
    Output("login-error-message", "children"),
    Output("dropdownAnios", "options"),
    Output("dropdownMeses", "options"),
    Output("dropdownTramos", "options"),
    Output("dropdownDistrito", "options"),
    Output("dropdownTipos", "options"),
    Output("dropdownModus", "options"),
    Output("dropdownCalificacion", "options"),
    Output("username-input", "value"),
    Output("password-input", "value"),
    Output("btn-usuario-cambiar-rol", "style"),
    Input("login-button", "n_clicks"),
    Input("elegir-usuario-button", "n_clicks"),
    Input("elegir-administrador-button", "n_clicks"),
    Input("btn-administrador-cambiar-rol", "n_clicks"),
    Input("btn-usuario-cambiar-rol", "n_clicks"),
    [
        State("username-input", "value"),
        State("password-input", "value"),
        State("btn-usuario-cambiar-rol", "style"),
    ],
)

def login(n_clicks, n_clicks_rol_usuario, n_clicks_rol_administrador, n_clicks_administrador_cambiar_rol, n_clicks_usuario_cambiar_rol, username, password, style_ir_admin):
    ctx = dash.callback_context
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    # Define default styles
    style_login = {"display": "none"}
    style_elegir_rol = {"display": "none"}
    style_usuario = {"display": "none"}
    style_administrador = {"display": "none"}
    mensaje_error_login = []
    dropdownAniosNuevasOpciones = []
    dropdownMesesNuevasOpciones = []
    dropdownTramosNuevasOpciones = []
    dropdownDistritoNuevasOpciones = []
    dropdownTiposNuevasOpciones = []
    dropdownModusNuevasOpciones = []
    dropdownCalificacionNuevasOpciones = []
    returnUser = ""
    returnPassword = ""
    new_style_ir_admin = style_ir_admin.copy()

    global user_logged
    
    if triggered_id == "login-button":
        result = log.comprobar_constraseña(username, password)
        if  result == 1:
            #Elegir rol visible
            style_elegir_rol = {"display": "block"}
            user_logged = username
        elif result == 0:
            #Rol usuario visible
            style_usuario = {"display": "block"}
            user_logged = username
            df = ruf.crear_dataframe()
            opciones_dropdown = ruf.generar_opciones_dropdown(df)
            dropdownAniosNuevasOpciones = opciones_dropdown[0]
            dropdownMesesNuevasOpciones = opciones_dropdown[1]
            dropdownTramosNuevasOpciones = opciones_dropdown[2]
            dropdownDistritoNuevasOpciones = opciones_dropdown[3]
            dropdownTiposNuevasOpciones = opciones_dropdown[4]
            dropdownModusNuevasOpciones = opciones_dropdown[5]
            dropdownCalificacionNuevasOpciones = opciones_dropdown[6]

        else:
            #Mostramos error en login
            style_login = {"display": "block"}
            mensaje_error_login = "Los datos introducidos no son correctos"

    elif triggered_id == "elegir-administrador-button":
        #Rol administrador visible
        style_administrador = {"display": "block"} 

    elif triggered_id == "elegir-usuario-button":
        #Rol usuario visible
        style_usuario = {"display": "block"}
        #Si venimos de pantalla de elegir rol, enseñamos boton cambiar a admin
        new_style_ir_admin["display"] = "block"
        df = ruf.crear_dataframe()
        opciones_dropdown = ruf.generar_opciones_dropdown(df)
        dropdownAniosNuevasOpciones = opciones_dropdown[0]
        dropdownMesesNuevasOpciones = opciones_dropdown[1]
        dropdownTramosNuevasOpciones = opciones_dropdown[2]
        dropdownDistritoNuevasOpciones = opciones_dropdown[3]
        dropdownTiposNuevasOpciones = opciones_dropdown[4]
        dropdownModusNuevasOpciones = opciones_dropdown[5]
        dropdownCalificacionNuevasOpciones = opciones_dropdown[6]

    elif triggered_id == "btn-administrador-cambiar-rol":
        #Rol usuario visible
        style_usuario = {"display": "block"}
        #Si venimos de pantalla de admin, enseñamos boton cambiar a admin
        new_style_ir_admin["display"] = "block"
        df = ruf.crear_dataframe()
        opciones_dropdown = ruf.generar_opciones_dropdown(df)
        dropdownAniosNuevasOpciones = opciones_dropdown[0]
        dropdownMesesNuevasOpciones = opciones_dropdown[1]
        dropdownTramosNuevasOpciones = opciones_dropdown[2]
        dropdownDistritoNuevasOpciones = opciones_dropdown[3]
        dropdownTiposNuevasOpciones = opciones_dropdown[4]
        dropdownModusNuevasOpciones = opciones_dropdown[5]
        dropdownCalificacionNuevasOpciones = opciones_dropdown[6]
    
    elif triggered_id == "btn-usuario-cambiar-rol":
        #Rol administrador visible
        style_administrador = {"display": "block"} 
    else:
        #Nos quedamos en login
        style_login = {"display": "block"}

    return style_login, style_usuario, style_administrador, style_elegir_rol, mensaje_error_login, dropdownAniosNuevasOpciones, dropdownMesesNuevasOpciones, dropdownTramosNuevasOpciones, dropdownDistritoNuevasOpciones, dropdownTiposNuevasOpciones, dropdownModusNuevasOpciones, dropdownCalificacionNuevasOpciones, returnUser, returnPassword, new_style_ir_admin

def dropdown_isNullOrEmpty(dropdown):
    return dropdown is None or len(dropdown) == 0

Creamos las funciones callback necesarias para rol usuario.

In [ ]:
# Con este callback controlamos los botones del navbar de usuario
# Controlamos la gestion de los filtros y la creacion de uno nuevo
@app.callback(
    Output("layout-usuario-visualizacion", "style"),
    Output("layout-usuario-gestion-filtros", "style"),
    Output("table-filtros", "data"),
    Output("table-filtros", "selected_rows"),
    Output("output-data-borrado-filtros", "children"),
    Output("dropdown-container-filtros", "style"),
    Output("dropdownFiltros", "options"),
    Output("crear-filtro-message", "children"),
    Output("centered-square-box", "style"),
    Output("filtro-input-container", "style"),
    Output("approved-box-button", "style"),
    Output("rejected-box-button", "style"),
    Output('lupa-container', 'style'),
    Input("btn-usuario-visualizacion", "n_clicks"),
    Input("btn-usuario-filtros", "n_clicks"),
    Input("borrar-filtros-button", "n_clicks"),
    Input("lupa-container", "n_clicks"),
    Input("crear-filtro-button", "n_clicks"),
    Input("crear-filtro-confirmacion-button", "n_clicks"),
    Input("close-box-button", "n_clicks"),
    [
        State("dropdownTramos", "value"),
        State("dropdownMeses", "value"),
        State("dropdownAnios", "value"),
        State("dropdownTipos", "value"),
        State("dropdownModus", "value"),
        State("dropdownCalificacion", "value"),
        State("dropdownDistrito", "value"),
        State("dropdownFiltros", "options"),
        State("centered-square-box", "style"),
        State("nombre-filtro-input", "value"),
        State('table-filtros', 'selected_rows'),
        State('table-filtros', 'data'),
        State('dropdown-container-filtros', 'style'),
        State('lupa-container', 'style')
    ]
)

def controlar_visibilidades_usuario_y_borrado_filtros(btn_usuario_visualizacion, btn_usuario_filtros, borrar_filtros_button, lupa_container_button, n_clicks_crear, n_clicks_confirmacion, n_clicks_close_box,
                                                      dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito, dropdownFiltrosOptions,
                                                      style_box, nombre_filtro, selected_rows, data_table, style_dropdown_filtro, style_lupa):
    # Determine which button was clicked
    ctx = dash.callback_context
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Define default styles
    style_visualizacion = {"display": "block"}
    style_filtros = {"display": "none"}
    filtros = []
    selected_rows_return = selected_rows
    borrado_filtros_texto = ""
    new_style_dropdown_filtro = style_dropdown_filtro
    dropdownFiltrosNuevasOpciones = dropdownFiltrosOptions
    crear_filtro_message = ""
    updated_style = style_box.copy()
    style_input = {"display": "block"}
    style_approved = {"display": "none"}
    style_rejected = {"display": "none"}
    new_style_lupa = style_lupa.copy()

    # Update styles based on the clicked button
    if triggered_id == "btn-usuario-visualizacion":
        style_visualizacion = {"display": "block"}
        new_style_lupa['display'] = "flex"
    
    if triggered_id == "crear-filtro-button":
        updated_style["display"] = "block"

        if dropdown_isNullOrEmpty(dropdownTramo) and dropdown_isNullOrEmpty(dropdownMes) and dropdown_isNullOrEmpty(dropdownAnio) and dropdown_isNullOrEmpty(dropdownTipo) and dropdown_isNullOrEmpty(dropdownModus) and dropdown_isNullOrEmpty(dropdownCalificacion) and dropdown_isNullOrEmpty(dropdownDistrito):
            crear_filtro_message = ["Seleccione algún filtro para poder guardar la combinación"]
            style_input = {"display": "none"}
            style_rejected = {"display": "block"}

    elif triggered_id == "crear-filtro-confirmacion-button":
        try:
            if nombre_filtro is None or len(nombre_filtro) == 0:
                crear_filtro_message = ["Escriba un nombre para poder guardar la combinación de filtros"]
            elif any(nombre_filtro == item['value'] for item in dropdownFiltrosOptions):
                crear_filtro_message = ["Ya existe una combinación de filtros con este nombre"]
            else:
                ruff.crear_filtro(user_logged, nombre_filtro, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)
                crear_filtro_message = ["El filtro ha sido creado con éxito"]
                style_input = {"display": "none"}
                
                style_approved = {"display": "block"}
                dropdownFiltrosNuevasOpciones = [ {"label": k, "value": k} for k in ruff.obtener_nombres_filtros_usuario(user_logged)]
        except Exception as e:
            print(e)
            traceback.print_exc()
            style_rejected = {"display": "block"}
            style_input = {"display": "none"}
            crear_filtro_message = ["Ha ocurrido un error al crear el filtro"]
        
    elif triggered_id == "close-box-button":
        updated_style["display"] = "none"

    elif triggered_id == "lupa-container":
        style_visualizacion = {"display": "block"}
        if new_style_dropdown_filtro['display'] == 'table':
                new_style_dropdown_filtro['display'] = 'none'
        else:
            new_style_dropdown_filtro['display'] = "table"
            dropdownFiltrosNuevasOpciones = [ {"label": k, "value": k} for k in ruff.obtener_nombres_filtros_usuario(user_logged)]

    elif triggered_id == "btn-usuario-filtros":
        style_filtros = {"display": "block"}
        style_visualizacion = {"display": "none"}
        new_style_lupa['display'] = "none"

        try:
            filtros = ruff.obtener_filtros_usuario(user_logged).to_dict('records')
            if filtros is None or len(filtros) == 0:
                borrado_filtros_texto = ["No dispone de filtros guardados"]

        except Exception as e:
            print(e)
            traceback.print_exc()
            borrado_filtros_texto = ["Ha ocurrido un error al obtener los filtros", html.Br(), "Inténtelo de nuevo"]

    elif triggered_id == "borrar-filtros-button":
        style_filtros = {"display": "block"}
        style_visualizacion = {"display": "none"}
        if selected_rows is None or len(selected_rows) == 0:
            borrado_filtros_texto = ["Debe seleccionar algún filtro a borrar"]
        else:
            try:
                selected_nombres = [data_table[i]['Nombre'] for i in selected_rows]
                ruff.borrar_filtros(user_logged, selected_nombres)
                selected_rows_return = []
                borrado_filtros_texto = ["Los filtros han sido borrados con éxito"]
                dropdownFiltrosNuevasOpciones = [ {"label": k, "value": k} for k in ruff.obtener_nombres_filtros_usuario(user_logged)]
            except Exception as e:
                print(e)
                traceback.print_exc()
                borrado_filtros_texto =  ["Ha ocurrido un error al borrar los filtros", html.Br(), "Inténtelo de nuevo"]

        try:
            filtros = ruff.obtener_filtros_usuario(user_logged).to_dict('records')
        except Exception as e:
            print(e)
            traceback.print_exc()
            borrado_filtros_texto =  ["Ha ocurrido un error al obtener los filtros", html.Br(), "Inténtelo de nuevo"]
    
    return style_visualizacion, style_filtros, filtros, selected_rows_return, borrado_filtros_texto, new_style_dropdown_filtro, dropdownFiltrosNuevasOpciones, crear_filtro_message, updated_style, style_input, style_approved, style_rejected, new_style_lupa

# Precargamos valores de dropdown cuando se selecciona un filtro guardado
@app.callback(
    [
        Output("dropdownTramos", "value"),
        Output("dropdownMeses", "value"),
        Output("dropdownAnios", "value"),
        Output("dropdownTipos", "value"),
        Output("dropdownModus", "value"),
        Output("dropdownCalificacion", "value"),
        Output("dropdownDistrito", "value"),
    ],
    Input("dropdownFiltros", "value"),
    [
        State("dropdownTramos", "options"),
        State("dropdownMeses", "options"),
        State("dropdownAnios", "options"),
        State("dropdownTipos", "options"),
        State("dropdownModus", "options"),
        State("dropdownCalificacion", "options"),
        State("dropdownDistrito", "options"),
    ]
)

def upload_and_write_data(dropdownFiltros, opcionesTramos, opcionesMeses, opcionesAnios, opcionesTipos, opcionesModus, opcionesCalificacion, opcionesDistrito):
    dropdownAnios = []
    dropdownMeses = []
    dropdownTramos = []
    dropdownDistrito = []
    dropdownTipos = []
    dropdownModus = []
    dropdownCalificacion = []
    if dropdownFiltros is not None:
        valores_guardados = ruff.obtener_valores_guardados_filtro(user_logged, dropdownFiltros)
        dropdownAnios = [value for value in  valores_guardados[0] if any(item['value'] == value for item in opcionesAnios)]
        dropdownMeses = [value for value in  valores_guardados[1] if any(item['value'] == value for item in opcionesMeses)]
        dropdownTramos = [value for value in  valores_guardados[2] if any(item['value'] == value for item in opcionesTramos)]
        dropdownDistrito = [value for value in  valores_guardados[3] if any(item['value'] == value for item in opcionesDistrito)]
        dropdownTipos = [value for value in  valores_guardados[4] if any(item['value'] == value for item in opcionesTipos)]
        dropdownModus = [value for value in  valores_guardados[5] if any(item['value'] == value for item in opcionesModus)]
        dropdownCalificacion = [value for value in  valores_guardados[6] if any(item['value'] == value for item in opcionesCalificacion)]

    return dropdownTramos, dropdownMeses, dropdownAnios, dropdownTipos, dropdownModus, dropdownCalificacion, dropdownDistrito

# Cargamos figuras y mapas cuando cambie algun dropdown
@app.callback(
    [
        Output("map", "children"),
        Output("output-data-1", "children"),
        Output("output-data-2", "children"),
        Output("output-data-3", "children"),
        Output("output-data-4", "children"),
        Output("output-data-message", "children"),
        Output("output-data-message-mapa", "children")
    ],
    [
        Input("dropdownTramos", "value"),
        Input("dropdownMeses", "value"),
        Input("dropdownAnios", "value"),
        Input("dropdownTipos", "value"),
        Input("dropdownModus", "value"),
        Input("dropdownCalificacion", "value"),
        Input("dropdownDistrito", "value"),
    ],
)

def upload_and_write_data(dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito):
    return ruf.generar_figuras(dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)

# Generacion de informe cuando pulsemos botón
@app.callback(
    Output("informe-output-message", "children"),
    Input("generar-informe-button", "n_clicks"),
    [
        State("dropdownTramos", "value"),
        State("dropdownMeses", "value"),
        State("dropdownAnios", "value"),
        State("dropdownTipos", "value"),
        State("dropdownModus", "value"),
        State("dropdownCalificacion", "value"),
        State("dropdownDistrito", "value"),
    ],
)

def generar_informe(n_clicks, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito):
    if n_clicks:
        try:
            df = ruf.crear_dataframe()
            return ruf.generar_informe_pdf(user_logged, df, n_clicks, dropdownTramo, dropdownMes, dropdownAnio, dropdownTipo, dropdownModus, dropdownCalificacion, dropdownDistrito)
        except Exception as e:
            print(e)
            traceback.print_exc()
            return ["Ha ocurrido un error al generar el informe", html.Br(), "Inténtelo de nuevo"]
    else: 
        return ""

Creamos las funciones callback necesarias para rol administrador.

In [ ]:
# Controlamos el cajetín para mostrar los archivos subidos
@app.callback(
    Output("output-data-not-upload", "children"),
    Input("upload-data", "filename"),
    Input("upload-data", "contents")
)

def upload_file(filename, contents):
    if filename is not None:
        divs = [
            html.Div([
                f"El archivo ¨{file}¨ ha sido subido.",
            ]) 
        for file in filename]


        return divs
    else:
        return html.Div(["Ningún archivo ha sido subido."])

# Controlamos el botón para añadir los archivos subidosS
@app.callback(
    Output("output-data-upload", "children"),
    Input("submit-button", "n_clicks"), 
    State("upload-data", "filename"),
    State("upload-data", "contents")
)

def upload_and_write_data(n_clicks, filenames, contents):
    try:
        if(n_clicks > 0):
            not_uploaded_files = []
            uploaded_files = []
            inserted_files = []
            listdenuncias = []
            if contents is not None:
                listdenuncias, listdenunciasYaInsertadas = raif.insert_data(contents)

                # Return a message indicating that the data was written to the database
                for index, file in enumerate(filenames):
                    if listdenunciasYaInsertadas[index] == 1 and listdenuncias[index] > 0:
                        inserted_files.append(file)
                    elif listdenuncias[index] == 0:
                        not_uploaded_files.append(file)
                    else:
                        uploaded_files.append(file)
                        
                divs = [html.Div([f"El archivo ¨{file}¨ ha sido resgitrado con éxito en la base de datos."]) for file in uploaded_files]
                divsYaInsertadas = [html.Div([f"El archivo ¨{file}¨ ya existía en la base de datos."]) for file in inserted_files]
                divs2 = [html.Div([f"El archivo ¨{file}¨ NO CONTIENE LAS CABECERAS ADECUADAS."]) for file in not_uploaded_files]
                return divs + divsYaInsertadas + divs2
    except pymysql.err.InterfaceError as e:
        print(e)
        traceback.print_exc()
        return html.Div(["Ha ocurrido un error al establecer la conexión con la base de datos, disculpe las molestias."])
    except Exception as e:
        print(e)
        traceback.print_exc()
        return html.Div(["Ha ocurrido un error al procesar los datos, disculpe las molestias."])    

# Con este callback controlamos los botones del navbar de administrador, la creacion de un usuario y el borrado de usuarios
# Ha sido necesario meter aqui la gestion de usuarios para que cuando se abrise la pestaña correspondiente se cargaran los usuarios
# La creacion tambien ha sido necesaria combinarla para la actualizacion de los usuarios al crear uno
# Borrar datos de varios años
@app.callback(
    Output("layout-administrador-subida-archivos", "style"),
    Output("layout-administrador-borrado-archivos", "style"),
    Output("layout-administrador-usuarios", "style"),
    Output("dropdownAnios-borrado", "options"),
    Output("table-usuarios", "data"),
    Output("table-usuarios", "selected_rows"),
    Output("output-data-borrado-usuarios", "children"),
    Output("output-data-usuarios", "children"),
    Output("output-data-borrado", "children"),
    Input("btn-administrador-insercion", "n_clicks"),
    Input("btn-administrador-borrado", "n_clicks"),
    Input("btn-administrador-usuarios", "n_clicks"),
    Input("borrar-usuarios-button", "n_clicks"),
    Input("crear-usuario-button", "n_clicks"),
    Input("borrar-datos-button", "n_clicks"),
    [
        State('table-usuarios', 'selected_rows'),
        State('table-usuarios', 'data'),
        State("crear-usuario-username-input", "value"),
        State("crear-usuario-password-input", "value"),
        State("crear-usuario-password-repetida-input", "value"),
        State("dropdownRol", "value"),
        State("dropdownAnios-borrado", "value")
    ]
)

def controlar_visibilidades(btn_insercion_clicks, btn_borrado_clicks, btn_usuarios_clicks, btn_borrado_usuarios_clicks, btn_crear_usuario_clicks, btn_borrado_anios_clicks,
                             selected_rows, data_table, username, password, passwordRepetida, administrador, dropdownAnios_values):
        # Determine which button was clicked
        ctx = dash.callback_context
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if triggered_id is None or triggered_id == '':
            return {"display": "block"}, {"display": "none"}, {"display": "none"}, [], [], selected_rows, "", "", ""
        else:
            # Define default styles
            style_insercion = {"display": "none"}
            style_borrado = {"display": "none"}
            style_usuarios = {"display": "none"}
            opciones_años = []
            usuarios = []
            usuarios_texto = ""
            selected_rows_return = selected_rows
            crear_usuarios_texto = ""
            borrar_años_texto = ""
            
            # Update styles based on the clicked button
            if triggered_id == "btn-administrador-insercion":
                style_insercion = {"display": "block"}
            elif triggered_id == "btn-administrador-borrado":
                style_borrado = {"display": "block"} 
                opciones_años = ragf.obtener_años_borrar()

            elif triggered_id == "btn-administrador-usuarios":
                style_usuarios = {"display": "block"}
                try:
                    usuarios = ragf.obtener_usuarios().to_dict('records')
                except Exception as e:
                    print(e)
                    traceback.print_exc()
                    usuarios_texto =  ["Ha ocurrido un error al obtener los usuarios", html.Br(), "Inténtelo de nuevo"]

            elif triggered_id == "crear-usuario-button":
                style_usuarios = {"display": "block"}
                selected_rows_return = []

                if(username is None or username == '' or password is None or passwordRepetida is None or administrador is None):
                    crear_usuarios_texto = "Rellene todos los campos"
                elif len(username) > 30:
                    crear_usuarios_texto = "El usuario debe tener menos de 30 caracteres"
                elif len(password) < 8:
                    crear_usuarios_texto = "La contraseña debe tener más de 8 caracteres"
                elif password != passwordRepetida:
                    crear_usuarios_texto = "Ambas contraseñas no coinciden"
                elif sum(1 for row in data_table if row['Usuario'] == username) > 0:
                    crear_usuarios_texto = "Ya existe un usuario con ese nombre"
                else:
                    try:
                        ragf.crear_usuario(username, password, administrador)
                        crear_usuarios_texto = f"El usuario '{username}' se ha regitrado con éxito"
                    except Exception as e:
                        print(e)
                        traceback.print_exc()
                        crear_usuarios_texto = ["Ha ocurrido un error al crear el usuario", html.Br(), "Inténtelo de nuevo"]
                    
                try:
                    usuarios = ragf.obtener_usuarios().to_dict('records')
                except Exception as e:
                    print(e)
                    traceback.print_exc()
                    usuarios_texto =  ["Ha ocurrido un error al obtener los usuarios", html.Br(), "Inténtelo de nuevo"]

            elif triggered_id == "borrar-usuarios-button":
                style_usuarios = {"display": "block"}
                try:
                    if selected_rows is None or len(selected_rows) == 0:
                        usuarios_texto = ["Debe seleccionar algún usuario a borrar"]
                    else:
                        administradores = sum(1 for row in data_table if row['Rol'] == 'Administrador')
                        administradores_selecionados = sum(1 for i in selected_rows if data_table[i]['Rol'] == 'Administrador')

                        if administradores - administradores_selecionados > 0:
                            selected_usuarios = [data_table[i]['Usuario'] for i in selected_rows]
                            ragf.borrar_usuarios(selected_usuarios)
                            selected_rows_return = []
                            usuarios_texto = ["Los usuarios han sido borrados con éxito"]
                        else:
                            selected_rows_return = []
                            usuarios_texto = ["No puede borrar todos los administradores."]

                    try:
                        usuarios = ragf.obtener_usuarios().to_dict('records')
                    except Exception as e:
                        print(e)
                        traceback.print_exc()
                        usuarios_texto =  [usuarios_texto, html.Br(), "Ha ocurrido un error al obtener los usuarios", html.Br(), "Inténtelo de nuevo"]

                except Exception as e:
                    print(e)
                    traceback.print_exc()
                    usuarios_texto =  ["Ha ocurrido un error al borrar los usuarios", html.Br(), "Inténtelo de nuevo"]

            elif triggered_id == "borrar-datos-button":
                style_borrado = {"display": "block"}
                if dropdownAnios_values is not None:
                    try:
                        ragf.borrar_datos(dropdownAnios_values)
                        opciones_años = ragf.obtener_años_borrar()
                        borrar_años_texto = ["Los datos han sido borrados con éxito"]
                    except Exception as e:
                        print(e)
                        traceback.print_exc()
                        borrar_años_texto = ["Ha ocurrido un error al borrar los datos", html.Br(), "Inténtelo de nuevo"]
                else:
                    borrar_años_texto = ["Escoja un año para borrar los datos"]
                
            return style_insercion, style_borrado, style_usuarios,  [{"label": k, "value": k} for k in opciones_años], usuarios, selected_rows_return, usuarios_texto, crear_usuarios_texto, borrar_años_texto
        
        

Por último, ejecutamos la aplicación

In [ ]:
app.run_server(port="8051")